In [6]:
# Road Trip Planner

import collections
import csv
import math
import random

random.seed(917)

FILENAME = 'usa_cities.csv'
# FILENAME = 'california.csv'
START_POINT = 'San Francisco, California'
# END_POINT = 'New York, New York'
END_POINT = 'San Diego, California'
KM_PER_DAY = 100
TRAVEL_MODE = 'ADVENTURE'  # pick from SHORTEST or ADVENTURE
BORING_FACTOR = 0.75  # 1 not boring, 0 super boring

def parse_dataset(reader):
    """Parses the raw CSV data into a dictionary of City namedtuples"""
    
    # Define namedtuple to store city information
    City = collections.namedtuple('City', ['name', 'lat', 'lon', 'county', 'state'])

    next(reader)  # skip first line, header
    
    database = {}
    for idx, line in enumerate(reader, start=2):
        try:
            name, county, state, lat, lon = line
            lat, lon = degrees_to_radians(lat), degrees_to_radians(lon)
        except:
            print(f'Could not parse file at line {idx}: {line}')
            continue
        else:
            city = City(name, lat, lon, county, state)
            database[(name, state)] = city

    if len(database) < 1:
        raise Exception('Could not parse *any* data from input file')

    return database


def retrieve_city_from_db(city_string, database):
    """Searches the database for a matching City
    
    'city_string' must be in the form "name, state".
    """
        
    name, state = map(str.strip, city_string.split(','))
    try:
        city = database[(name, state)]
    except KeyError:
        raise Exception(f'City not found in database: {name}, {state}')
    else:
        print(f'Found match for {name}, {state}: {city.name}, {city.state}')
        return city


def degrees_to_radians(value):
    """Converts a lat/lon value in degrees to radians"""
    return float(value) * 3.14 / 180


def haversine_distance(city_i, city_j):
    """Returns the distance between two cities using the haversine formula"""

    lat_i, lon_i = city_i.lat, city_i.lon
    lat_j, lon_j = city_j.lat, city_j.lon

    # Haversine formula for distances between points on a sphere
    # https://en.wikipedia.org/wiki/Haversine_formula
    dlat = lat_j - lat_i
    dlon = lon_j - lon_i
    sin_dlat_h = math.sin(dlat/2)
    sin_dlon_h = math.sin(dlon/2)
    
    a = (sin_dlat_h*sin_dlat_h) + math.cos(lat_i) * math.cos(lat_j) * (sin_dlon_h*sin_dlon_h)
    c = 2 * math.atan2( math.sqrt(a), math.sqrt(1-a) )
    d = 6373  * c  # R is 'a' radius of earth
    
    return d


def get_city_neighbors(query, database, radius=500):
    """Returns neighbors of a given city"""
    
    return [city for city in database.values() 
            if haversine_distance(city, query) <= radius]

#     neighbors = []
#     for city in database.values():
#         if haversine_distance(city, query) <= radius:
#             neighbors.append(city)
#     return neighbors

#     is_close = lambda c: haversine_distance(c, query) <= radius
#     return filter(is_close, database.values())


def take_shortest_route(neighbors, route, end_point):
    """Picks the next stop in the route by calculating distances
    between cities and the end point and choosing the shortest"""
    
    d_to_end = lambda c: haversine_distance(c, end_point)
    
    sorted_neighbors = sorted(neighbors, key=d_to_end)
    for city in sorted_neighbors:
        if city not in route:
            break
    else:
        raise Exception('Route is not possible with current parameters')

    return city

#     sorted_neighbors = sorted(neighbors, key=d_to_end)
#     for city in sorted_neighbors:
#         if city not in route:
#             return city

#     raise Exception('Route is not possible with current parameters')


def take_adventure_route(neighbors, route, end_point, boringness_factor=BORING_FACTOR):
    """Picks the next stop somewhat randomly"""
    
    if random.random() >= boringness_factor:
        return take_shortest_route(neighbors, route, end_point)
    
    random.shuffle(neighbors)
    for city in neighbors:
        if city not in route:
            return city

    raise Exception('Adventure mode failed. Try a more boring route.')


def display_route(route):
    """Pretty prints a route"""
    
    cities = list(route.keys())

    tot_distance = 0.0

    print(f'Suggested Route ({len(cities)} stops):')
    for idx, stop in enumerate(cities[1:], start=1):

        start = cities[idx - 1]
        d = haversine_distance(start, stop)
        tot_distance += d
        
        print((f'Day #{idx:<3d} [{d:>5.1f} km]: '
               f'{start.name}, {start.state} -> {stop.name}, {stop.state}'))
        


    print(f'Total distance: {tot_distance:6.1f} km')
    print(f'Average: {tot_distance/len(route):>4.1f} km per day')
    

# Define search modes
picker = {'SHORTEST': take_shortest_route,
          'ADVENTURE': take_adventure_route}
#
# Main Code
#

pick_next_city = picker.get(TRAVEL_MODE)
if pick_next_city is None:
    raise Exception(f"Travel mode '{TRAVEL_MODE}' not supported.")

with open(FILENAME, encoding='utf-8') as handle:
    reader = csv.reader(handle)   
    city_dict_db = parse_dataset(reader)

print(f'Read {len(city_dict_db)} cities from database')

sp_city = retrieve_city_from_db(START_POINT, city_dict_db)
ep_city = retrieve_city_from_db(END_POINT, city_dict_db)

# Find 'best' route
route = collections.OrderedDict()
route = route.fromkeys([sp_city])

current_city = sp_city
while current_city != ep_city:
    neighbors = get_city_neighbors(current_city, city_dict_db, radius=KM_PER_DAY)
    current_city = pick_next_city(neighbors, route, ep_city)
    route[current_city] = None

display_route(route)

Read 29738 cities from database
Found match for San Francisco, California: San Francisco, California
Found match for San Diego, California: San Diego, California
Suggested Route (48 stops):
Day #1   [ 97.9 km]: San Francisco, California -> Gilroy, California
Day #2   [ 56.8 km]: Gilroy, California -> Gustine, California
Day #3   [ 88.7 km]: Gustine, California -> Moccasin, California
Day #4   [ 94.2 km]: Moccasin, California -> O Neals, California
Day #5   [ 31.2 km]: O Neals, California -> Bass Lake, California
Day #6   [ 89.9 km]: Bass Lake, California -> Orosi, California
Day #7   [ 81.3 km]: Orosi, California -> San Joaquin, California
Day #8   [ 62.2 km]: San Joaquin, California -> Friant, California
Day #9   [ 75.5 km]: Friant, California -> Merced, California
Day #10  [ 63.9 km]: Merced, California -> Modesto, California
Day #11  [ 91.9 km]: Modesto, California -> Union City, California
Day #12  [ 49.5 km]: Union City, California -> San Pablo, California
Day #13  [ 75.5 km]: San